In [21]:
#This version of the evolutionary algorithm seeks to implement
#elitism, i.e. the keeping of the best solution from previous generations
#and step 6, adding factories from focuses and annexations (mainly the latter)
#add a baseline number of trade factories

#Note: I only added the factories from the baltics, as the other territories
#have little industry
#Also note that factories from annexed territories increase over time as compliance
#grows, this is something that I have not modelled and likely accounts for the 
#remaining underestimation of factories
#Finally, the amount of IC produced is actually higher than they should be
#This is because of problems with the production efficiency calculations

In [22]:
#Initalize an array of tuples, indicating how much production went into them and their type
#Keep track of the amount of construction done by all civs
#Keep track of how many available civs there are, give each entry a max of 15 civs worth of production
#At the start of a new state:
#If there is no production in a slot change it to the factory the model wants to build
#If a factory is >5% finished continue building it
#Production overflow is saved and added to the first unfinished factory in the array afterwards


In [23]:
#Production efficiency techs take 150 days base, construction and industry take 200 base

#Meta strat:
#21 march 1936: Improved worker conditions
#2 september 1936: War eco
#27 november 1936: Popular figurehead
#30 march 1937: Captain of industry, state 12


#Normal strat:
#17 july 1936: Spanish civil war, war eco


#Research:
#23 may 1936: Basic machine tools, state 4
#8 july 1936: Construction 1, state 5
#24 november 1936: Concentrated 1, state 9 
#28 feburari 1937: Construction 2, state 12
#11 april 1937: Improved machine tools, state 13
#25 may 1937: Concentrated 2, state 14

#ASSUMPTION FOR NOW: All further tech is unlocked 3 months after they become current TEST IN GAME
#Concentrated 3, Machine tools 3, Construction 3, state 33
#Concentrated 4, Machine tools 4, Construction 4, state 54

In [24]:
#STABILITY EVENTS:
#State 2: +5%
#State 4: +5%
#State 8: +10%

In [25]:
import numpy as np
import random


In [26]:
def selection(pop, scores, k=3):
    selection_ix = np.random.randint(len(pop))
    for ix in np.random.randint(0, len(pop), k-1):
        if scores[ix] > scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if np.random.rand() < r_cross:
        pt = np.random.randint(1, len(p1)-1)
        #print(pt)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
        #visualise what solutions have been tried
    return [c1, c2]

def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if np.random.rand() < r_mut:
            bitstring[i] = np.random.randint(0, 4)#1 - bitstring[i]
    return bitstring

In [27]:
#mutation([1] * 10 + [0] * 10, 1/4)

In [28]:
def update_consumergoods(state, toaster_percent):
    match state:
        case 7:
            toaster_percent -= 0.15 #WAR ECO     
        case 30:
            toaster_percent -= 0.05 #construction company
        case 34:
            toaster_percent -= 0.03 #stability bonus
    return round(toaster_percent, 2)

def update_civs(state, civs):
    match state:
        case 8:
            civs += 2
        case 20:
            civs += 2
        case 22:
            civs += 2
        case 25:
            civs += 2
        case 34:
            civs += 13 #annexation of baltics        
    return civs

def update_mils(state, mils):
    match state:
        case 34:
            mils += 8    
    return mils


def update_constructionspeed(state, civ_output_modifier, scenario):
    match scenario:
        case 'one year':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 12: #same as half year because we start in 36
                    civ_output_modifier += 0.1 
                case 31:
                    civ_output_modifier += 0.1
                case 52:
                    civ_output_modifier += 0.1
        case 'half year':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 12: 
                    civ_output_modifier += 0.1 
                case 33:
                    civ_output_modifier += 0.1
                case 54:
                    civ_output_modifier += 0.1
        case 'current':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 16: 
                    civ_output_modifier += 0.1 
                case 36:
                    civ_output_modifier += 0.1
                case 57:
                    civ_output_modifier += 0.1
            
    return round(civ_output_modifier, 2)

def update_miloutput(state, mil_output_modifier, scenario):
    match state: #These are the bonuses related to stability
        case 0:
            mil_output_modifier -= 0.2180
        case 2: 
            mil_output_modifier += 0.05
        case 4: 
            mil_output_modifier += 0.05
        case 8:
            mil_output_modifier += 0.10
        case 9: #At this point we are above 50% stability
            mil_output_modifier += 0.04
        case 10: #popular figurehead
            mil_output_modifier += 0.06 
        case 24: #kill trotsky + new NKVD head
            mil_output_modifier += 0.04
    
    match scenario:
        case 'one year':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 14: 
                    mil_output_modifier += 0.15
                case 31:
                    mil_output_modifier += 0.15
                case 52:
                    mil_output_modifier += 0.15
        case 'half year':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 14: 
                    mil_output_modifier += 0.15
                case 33:
                    mil_output_modifier += 0.15
                case 54:
                    mil_output_modifier += 0.15
        case 'current':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 16: 
                    mil_output_modifier += 0.15
                case 36:
                    mil_output_modifier += 0.15
                case 57:
                    mil_output_modifier += 0.15

    return round(mil_output_modifier, 3)

def update_prodefficiency(state, production_efficiency_cap, scenario):
    match state: #These are related to the 'defense industry' buff in the focus tree
        case 30:
            production_efficiency_cap += 0.05
        case 40:
            production_efficiency_cap += 0.05
    
    match scenario:
        case 'one year':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 13:
                    production_efficiency_cap += 0.1 
                case 26: #construction company american experts
                    production_efficiency_cap += 0.1 
                case 31:
                    production_efficiency_cap += 0.1
                case 52:
                    production_efficiency_cap += 0.1
        case 'half year':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 13: 
                    production_efficiency_cap += 0.1 
                case 26: #construction company american experts
                    production_efficiency_cap += 0.1 
                case 33:
                    production_efficiency_cap += 0.1
                case 54:
                    production_efficiency_cap += 0.1
        case 'current':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 15: 
                    production_efficiency_cap += 0.1 
                case 26: #construction company american experts
                    production_efficiency_cap += 0.1 
                case 36:
                    production_efficiency_cap += 0.1
                case 57:
                    production_efficiency_cap += 0.1

    return round(production_efficiency_cap, 2)

def update_civcost(state, civ_construction_modifier):
    match state:
        case 0:
            civ_construction_modifier += 0.05 #five year plan
        case 7:
            civ_construction_modifier += 0.3
        case 12: 
            civ_construction_modifier += 0.1 #captain of industry
        case 21:
            civ_construction_modifier += 0.05 #construction company
        case 30:
            civ_construction_modifier += 0.05 #construction company upgrade
    return round(civ_construction_modifier, 2)

def update_milcost(state, mil_construction_modifier):
    match state:
        case 7:
            mil_construction_modifier += 0.5
        case 21:
            mil_construction_modifier += 0.05 #construction company
    return round(mil_construction_modifier, 2)

def update_civ_to_milcost(state, civ_to_mil_construction_modifier):
    match state:
        case 7:
            civ_to_mil_construction_modifier += 0.5
    return round(civ_to_mil_construction_modifier, 2)

def update_mil_to_civcost(state, mil_to_civ_construction_modifier):
    match state:
        case 7:
            mil_to_civ_construction_modifier += 0.5
    return round(mil_to_civ_construction_modifier, 2)

def construction_queue_update(construction_queue, action, available_civs, civ_output, state_size_days,
                              infrastructure_modifier, civ_output_modifier, civ_construction_modifier,
                              mil_construction_modifier, civ_to_mil_construction_modifier, 
                             mil_to_civ_construction_modifier):
    free_civs = available_civs
    for i in range(len(construction_queue)):
        if construction_queue[i] == None:
            construction_queue[i] = (0,0)
        
    for x in range(len(construction_queue)):  
        if action == 0 and construction_queue[x][0] == 0:
            construction_queue[x] = (0, 7200, 'military')
        if action == 1 and construction_queue[x][0] == 0:
            construction_queue[x] = (0, 10800, 'civilian')
        if action == 2 and construction_queue[x][0] == 0:
            construction_queue[x] = (0, 4000, 'civ to mil')
        if action == 3 and construction_queue[x][0] == 0:
            construction_queue[x] = (0, 9000, 'mil to civ')
    
    #Calculate how much a single factory would contribute to either of the four actions
    #during one state
    build_civ_output = civ_output * state_size_days * infrastructure_modifier \
    * civ_output_modifier * civ_construction_modifier
    
    build_mil_output = civ_output * state_size_days * infrastructure_modifier \
    * civ_output_modifier * mil_construction_modifier
    
    build_civ_to_mil_output = civ_output * state_size_days * infrastructure_modifier \
    * civ_output_modifier * civ_to_mil_construction_modifier
    
    build_mil_to_civ_output = civ_output * state_size_days * infrastructure_modifier \
    * civ_output_modifier * mil_to_civ_construction_modifier
    
    #print(build_civ_output, build_mil_output, build_civ_to_mil_output, build_mil_to_civ_output)
    for j in range(len(construction_queue)):
        if free_civs > 0 and free_civs < 15:
            print(construction_queue[j][2])
            
            free_civs -= free_civs 
            print(j, free_civs, available_civs)
        if free_civs > 0:
            
            
            free_civs -= 15 
            print(j, free_civs, available_civs)            
        else:
            break
    
    return construction_queue

def prod_eff_calculations(state, prod_eff, prod_eff_cap, factory_state_array, new_mils):
    factory_count_list = []
    efficiency_list = []
    sum_of_efficiency = 0
    prod_eff = 0
    factory_state_array[state][0] = new_mils
    factory_state_array[state][1] = 0.1
    #factory_count += (factory_state_array[state][0])
    #print(factory_count)
    if state == 0:
        factory_state_array[state][1] = 0.5
    
    for i in range(state):
        factory_count_list.append(factory_state_array[i][0])
        efficiency_list.append(factory_state_array[i][1])   
        #if factory_state_array[i][0] > 0 and factory_state_array[i][1] < min(prod_eff_cap, 1):

        growth = 0.001 * (prod_eff_cap ** 2 / factory_state_array[i][1]) * 17.5
        factory_state_array[i][1] += growth
        factory_state_array[i][1] = min(factory_state_array[i][1], 1)
        
    num_factories = sum(factory_count_list)
    
    #print(efficiency_list)
    for j in range(len(factory_count_list)):
        sum_of_efficiency += factory_count_list[j] * efficiency_list[j]
    
    try:
        return min(sum_of_efficiency / num_factories, 1)
    except:
        return 0.5 #This should only happen at the start, then eff = 0.5
            
    #print(factory_state_array)
        #print(i, factory_state_array[state][0], factory_count)
    
    #return factory_state_array

In [29]:
#Four max_targets: 'rush', 'scale', 'scale_total' and 'scale_civs'

def production(actions, max_target, scenario):
    
    #ic_prod_list = []
    mils = 32
    mils_0 = 32 #keep track of the number of starting factories
    new_mils = 32 #initialize production efficiency
    civs = 45 + 8 #(a baseline number of factories from trade)
    civs_0 = 45 #idem
    total = mils + civs
    total_0 = 32 + 45 #idem
    consumergoods_percentage = 0.40
    consumergoods = np.floor(total * consumergoods_percentage)
    available_civs = np.maximum(int(civs - consumergoods), 0)
    
    #Civ output modifiers are additive (-30% + 15% = -15%), multiplied by state infrastructure
    civ_output = 5 
    civ_output_modifier = 1 #Construction speed
    civ_construction_modifier = 0.7
    mil_construction_modifier = 0.7
    civ_to_mil_construction_modifier = 0.7
    mil_to_civ_construction_modifier = 0.7
    
    mil_output = 4.5
    mil_output_modifier = 1
    prod_eff = 0.5
    production_efficiency_cap = 0.55
    
    #Investigate how new factories add to production efficiency
    #production_efficiency works as on the wiki, except for the starting factories
    #with different rules
    #The number of factories because of trade is quite significant, but ultimately unpredictable
    
    state = 0
    state_size_days = 35 #how many days per state
    ic_produced = 0
    ic_total = 0    
    civ_purchase_pool = 0 
    mil_purchase_pool = 0
    civ_to_mil_pool = 0
    mil_to_civ_pool = 0
    infrastructure_modifier = 1.6
    
    factory_state_array = np.empty((57), dtype=object)

    for i in range(len(factory_state_array)):
        factory_state_array[i] = np.zeros(2)

    #construction_queue = np.zeros(30) 
    #construction_queue = np.empty((30), dtype=object) #equates to 450 free civs, more than will be reached in practice
    
    #print(construction_queue_update(construction_queue, 0))
    #construction_queue = construction_queue_update(construction_queue, 1, available_civs, civ_output, state_size_days,
    #                          infrastructure_modifier, civ_output_modifier, civ_construction_modifier,
    #                          mil_construction_modifier, civ_to_mil_construction_modifier, 
    #                         mil_to_civ_construction_modifier)
    #print(construction_queue)
    
    for i in range(len(actions)):
        
        civs = (update_civs(i, civs))
        mils = (update_mils(i, mils))
        consumergoods_percentage = (update_consumergoods(i, consumergoods_percentage))
        available_civs = np.maximum(int(civs - consumergoods), 0)
        civ_output_modifier = (update_constructionspeed(i, civ_output_modifier, scenario))
        mil_output_modifier = (update_miloutput(i, mil_output_modifier, scenario))
        civ_construction_modifier = (update_civcost(i, civ_construction_modifier))
        mil_construction_modifier = (update_milcost(i, mil_construction_modifier))
        civ_to_mil_construction_modifier = (update_civ_to_milcost(i, civ_to_mil_construction_modifier))
        mil_to_civ_construction_modifier = (update_mil_to_civcost(i, mil_to_civ_construction_modifier))
        production_efficiency_cap = (update_prodefficiency(i, production_efficiency_cap, scenario))
        
        ic_produced = mil_output * mils * state_size_days * mil_output_modifier * prod_eff
        #ic_prod_list.append(ic_produced)
        #print(("available civs: " + str(available_civs), "consumer goods: " + str(consumergoods_percentage), 
        #       "civs: " + str(civs), "mils: " + str(mils)))
        prod_eff = prod_eff_calculations(i, prod_eff, production_efficiency_cap, factory_state_array, new_mils)
        #print(prod_eff)

        if actions[i] == 0: #build mils     
            mil_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * mil_construction_modifier)
            existing_mils = mils #Used exclusively for production efficiency calculations
            new_mils = np.floor_divide(mil_purchase_pool, 7200) #for production efficiency
            mils += np.floor_divide(mil_purchase_pool, 7200) 
            total += np.floor_divide(mil_purchase_pool, 7200) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            mil_purchase_pool = np.mod(mil_purchase_pool, 7200)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            
            #prod_eff = min(1, (existing_mils * prod_eff + (mils - existing_mils) * 0.1) / mils)            
            #growth = 0.001 * (production_efficiency_cap ** 2 / prod_eff)
            #prod_eff += (growth * state_size_days) / 2
            #prod_eff = min(1, mil_output_modifier)            
            state += 1
            ic_total += ic_produced
            
        if actions[i] == 1: #build civs         
            civ_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_construction_modifier)
            #construction_queue = construction_queue_update(construction_queue, actions[i], available_civs, civ_output, state_size_days,
            #                  infrastructure_modifier, civ_output_modifier, civ_construction_modifier,
            #                  mil_construction_modifier, civ_to_mil_construction_modifier, 
            #                  mil_to_civ_construction_modifier)
            new_mils = 0 #Account for annexations here
            civs += np.floor_divide(civ_purchase_pool, 10800) 
            total += np.floor_divide(civ_purchase_pool, 10800) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            civ_purchase_pool = np.mod(civ_purchase_pool, 10800)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            
            #growth = 0.001 * (production_efficiency_cap **2 / prod_eff)
            #prod_eff += (growth * state_size_days) / 2
            #prod_eff = min(1, prod_eff)
            
            state += 1
            ic_total += ic_produced

        if actions[i] == 2: #Convert civs to mils
            civ_to_mil_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_to_mil_construction_modifier)
            new_mils = np.floor_divide(civ_to_mil_pool, 4000)
            civs -= np.floor_divide(civ_to_mil_pool, 4000)
            mils += np.floor_divide(civ_to_mil_pool, 4000)
            civ_to_mil_pool = np.mod(civ_to_mil_pool, 4000)
            consumergoods = np.ceil(total * consumergoods_percentage)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            ic_total += ic_produced
        
        if actions[i] == 3: #Convert mils to civs
            mil_to_civ_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * mil_to_civ_construction_modifier)
            if mils > 0:
                new_mils = 0
                civs += np.floor_divide(mil_to_civ_pool, 9000)
                mils -= np.floor_divide(mil_to_civ_pool, 9000)
                
                if mils < 0:
                    too_many_removed = abs(mils)
                    civs -= too_many_removed
                    mils = 0
                    mil_to_civ_pool = 0
                
                else:
                    mil_to_civ_pool = np.mod(mil_to_civ_pool, 9000)
                consumergoods = np.ceil(total * consumergoods_percentage)
                available_civs = np.maximum(int(civs - consumergoods), 0)
            ic_total += ic_produced
            
    match max_target:
        case 'rush':
            return ((ic_total, mils, civs))
        case 'scale':
            scaled_score = ic_total * np.log2((mils/mils_0))
            return ((scaled_score, mils, civs, ic_total))
        case 'scale_total':
            scaled_score = ic_total * np.log2((mils/mils_0)) * (total/total_0)
            if scaled_score == None:
                scaled_score = 0
            return ((scaled_score, mils, civs, ic_total))
        case 'scale_civs':
            scaled_score = ic_total * np.log2((mils/mils_0)) * np.log2((civs/civs_0))
            if scaled_score == None:
                scaled_score = 0
            return ((scaled_score, mils, civs, ic_total))
        case 'mils':
            return ((mils, civs, ic_total))

In [30]:
#test = ([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
test = [1] * 36 + [0] * 21
#test = [0] * 57
production(test, 'scale', 'half year')


(1479898.3066962948, 191.0, 145.0, 574176.2063801084)

In [31]:
test = [0] * 57
test2 = [1] * 16 + [0] * 41
test3 = [1] * 57

production(test, 'rush', 'half year'), production(test2, 'rush', 'half year'), production(test3, 'rush', 'half year')

#production(test, 'scale'), production(test2, 'scale'), production(test3, 'scale')

#production(test, 'scale_total'), production(test2, 'scale_total'), production(test3, 'scale_total')

#production(test, 'scale_civs'), production(test2, 'scale_civs'), production(test3, 'scale_civs')

((791767.0442336968, 162.0, 74),
 (750434.1166438893, 175.0, 88.0),
 (379047.50497067336, 40, 313.0))

In [32]:
def genetic_algorithm(n_bits, n_pop, n_iter, r_cross, max_target, scenario):

    pop = [np.random.randint(0, 4, n_bits).tolist() for _ in range(n_pop)]
    best, best_eval = 0, production(pop[0], max_target, scenario)
    
    for gen in range(n_iter):
        scores = [production(c, max_target, scenario) for c in pop]
        for i in range(n_pop):
            if scores[i] > best_eval:
                best, best_eval = pop[i], scores[i]
                print("Generation " + str(gen) + ": new best solution found!")
                print(best, best_eval)
                
        #ADD ELITISM PLUS ADDITIONAL COMPLETELY RANDOM SAMPLES HERE
        
        selected = [selection(pop, scores) for _ in range(n_pop)]
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            c = crossover(p1, p2, r_cross)
            for child in c:
                mutated = mutation(child, 1 / n_bits)
                children.append(mutated)
        #print(np.mean(sum([production(c) for c in pop])))
        #print(np.mean(sum([production(c) for c in children])))
        #print(sum([sum(strat) for strat in pop]))
        del children[-1] #Delete the last element and replace with best solution
        children.append(best)
        pop = children
        
    #SMOOTHING TEST
    best_smoothed = [best[0]]
    for i in range(1, len(best)-1):
        #print((best[i-1],best[i],best[i+1]))
        if best[i-1] == best[i+1]:
            best_smoothed.append(best[i-1])
        else:
            best_smoothed.append(best[i])
    best_smoothed.append(best[-1])
    best_smoothed_eval = production(best_smoothed, max_target, scenario)
    if best_smoothed_eval > best_eval:
        print("Smoothing : new best solution found!")
        print(best_smoothed, best_smoothed_eval)
        return [best_smoothed, best_smoothed_eval]
    
    return [best, best_eval]

In [33]:
genetic_algorithm(10, 10, 10, 0.9, 'rush', 'half year') #There should be 66 states of 30 days, ends up on June 5th '41
#Note: states could also be 35 days to reflect the length of the focus tree
#This would be 57 states of 35 days, ending onJune 19th '41 (no leap years)

Generation 0: new best solution found!
[2, 0, 3, 0, 1, 3, 0, 1, 2, 1] (24978.531106082362, 37.0, 52.0)
Generation 0: new best solution found!
[0, 2, 0, 2, 1, 0, 0, 3, 2, 1] (25255.10089886008, 39.0, 51.0)
Generation 0: new best solution found!
[2, 0, 2, 0, 1, 2, 2, 0, 2, 0] (25337.074745140882, 41.0, 49.0)
Generation 2: new best solution found!
[0, 2, 2, 2, 2, 2, 1, 2, 1, 1] (25415.335241931705, 38.0, 51.0)
Generation 3: new best solution found!
[2, 0, 2, 0, 2, 2, 1, 2, 1, 1] (25449.08242478765, 38.0, 52.0)
Generation 3: new best solution found!
[0, 2, 2, 2, 1, 2, 2, 0, 2, 0] (25487.08830318165, 41.0, 48.0)
Generation 4: new best solution found!
[0, 2, 0, 2, 2, 2, 1, 2, 2, 1] (25753.22841410924, 41.0, 48.0)
Generation 5: new best solution found!
[0, 2, 0, 2, 2, 2, 1, 2, 2, 2] (25753.22841410924, 43.0, 45.0)
Generation 7: new best solution found!
[0, 2, 0, 2, 2, 2, 2, 2, 2, 2] (25817.040459637334, 44.0, 44.0)


[[0, 2, 0, 2, 2, 2, 2, 2, 2, 2], (25817.040459637334, 44.0, 44.0)]

In [34]:
#smoothing does not bring better results anymore, at least for 'scale' max_target
#the results for 'scale' seem very similar to those of 'rush'
#It produces 96% of the IC, has 6% more mils and 13% more civs
genetic_algorithm(57,100, 60, 0.9, 'scale', 'half year')

Generation 0: new best solution found!
[3, 2, 1, 2, 2, 0, 2, 1, 1, 1, 1, 1, 2, 2, 2, 0, 1, 0, 2, 2, 0, 1, 3, 3, 1, 1, 0, 3, 0, 2, 0, 3, 2, 2, 1, 0, 0, 0, 0, 0, 0, 3, 0, 1, 2, 1, 3, 3, 0, 3, 3, 2, 1, 2, 2, 1, 0] (1064810.8753869096, 108.0, 58.0, 606768.7382092496)
Generation 0: new best solution found!
[0, 1, 0, 0, 2, 3, 2, 2, 3, 2, 3, 0, 1, 0, 2, 0, 2, 0, 3, 1, 1, 0, 2, 1, 0, 2, 3, 1, 1, 1, 0, 2, 2, 0, 2, 1, 2, 1, 2, 0, 0, 0, 2, 3, 2, 3, 2, 0, 1, 2, 2, 2, 0, 0, 2, 0, 0] (1207419.534305729, 117.0, 36.0, 645552.9884966305)
Generation 0: new best solution found!
[1, 2, 3, 1, 3, 1, 1, 0, 0, 0, 2, 0, 1, 2, 1, 3, 1, 1, 1, 1, 1, 3, 0, 0, 2, 1, 0, 3, 2, 2, 0, 1, 1, 1, 2, 0, 1, 2, 3, 2, 0, 2, 3, 1, 0, 1, 2, 1, 0, 0, 0, 1, 0, 2, 1, 2, 2] (1210837.217142554, 123.0, 51.0, 623334.9680604284)
Generation 0: new best solution found!
[2, 2, 0, 3, 2, 3, 0, 2, 3, 1, 3, 1, 0, 1, 0, 1, 3, 1, 0, 1, 0, 0, 2, 1, 0, 1, 3, 3, 3, 0, 1, 1, 0, 3, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 1, 2, 0, 3, 3, 0]

Generation 18: new best solution found!
[1, 1, 0, 0, 3, 3, 1, 0, 0, 1, 0, 0, 1, 3, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0] (1676302.1826117432, 165.0, 50.0, 708399.799706063)
Generation 18: new best solution found!
[1, 1, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 2, 2, 1, 0, 1, 0, 2, 2, 2, 0] (1695174.8598823394, 163.0, 62.0, 721741.6186835392)
Generation 18: new best solution found!
[1, 1, 0, 1, 3, 1, 1, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2] (1696784.1304552348, 159.0, 44.0, 733622.9992004179)
Generation 18: new best solution found!
[1, 1, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 1, 0, 1, 0, 2, 2, 

[[1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  2,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2],
 (1880277.2976570833, 170.0, 47.0, 780395.2731187752)]

In [35]:
#The best strategy for 'scale_total' produces about 81% percent of the IC that the rush strategy does
#It ends up with 18% extra mils and 66% extra civs though
genetic_algorithm(57,100, 60, 0.9, 'scale_total', 'half year')

Generation 0: new best solution found!
[3, 3, 2, 0, 2, 3, 0, 3, 1, 1, 1, 1, 1, 3, 0, 0, 1, 3, 1, 0, 2, 1, 3, 3, 3, 1, 3, 2, 0, 3, 2, 1, 3, 0, 0, 1, 3, 0, 0, 1, 0, 3, 1, 2, 1, 1, 0, 1, 2, 0, 3, 1, 0, 2, 2, 3, 3] (1941283.7666967085, 107.0, 95.0, 496156.1731166464)
Generation 0: new best solution found!
[3, 1, 0, 3, 0, 2, 2, 1, 0, 0, 1, 0, 0, 1, 3, 3, 3, 1, 0, 0, 1, 1, 2, 1, 0, 0, 1, 0, 1, 0, 3, 3, 3, 3, 0, 1, 3, 0, 0, 0, 1, 2, 2, 3, 3, 2, 3, 1, 2, 2, 2, 2, 1, 3, 0, 2, 0] (2218224.1404908895, 123.0, 65.0, 553012.2576870066)
Generation 0: new best solution found!
[1, 1, 1, 3, 3, 2, 0, 0, 2, 2, 0, 1, 1, 0, 1, 1, 1, 2, 0, 0, 1, 0, 2, 2, 1, 3, 0, 1, 1, 1, 3, 3, 0, 0, 0, 2, 0, 1, 1, 0, 3, 3, 1, 1, 2, 1, 0, 1, 0, 0, 0, 3, 2, 1, 1, 2, 1] (2263900.2132824543, 112.0, 77.0, 602815.7305161873)
Generation 1: new best solution found!
[1, 1, 1, 3, 3, 2, 0, 0, 2, 2, 0, 1, 1, 0, 1, 1, 1, 2, 0, 0, 1, 0, 2, 2, 1, 3, 0, 1, 1, 1, 3, 3, 0, 0, 0, 2, 0, 1, 1, 0, 3, 3, 1, 1, 2, 1, 0, 1, 0, 0, 0, 3, 2, 1, 1, 1, 

Generation 16: new best solution found!
[2, 0, 1, 1, 1, 3, 0, 0, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 0, 1, 3, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 2] (4841458.976785176, 174.0, 95.0, 635831.9070244925)
Generation 17: new best solution found!
[3, 0, 1, 0, 0, 3, 3, 3, 0, 3, 1, 1, 3, 1, 1, 1, 1, 3, 0, 1, 3, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 2] (4862770.616101725, 176.0, 98.0, 621403.4677393885)
Generation 17: new best solution found!
[3, 0, 1, 1, 1, 3, 0, 0, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 0, 1, 3, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 2] (4987602.795399879, 177.0, 98.0, 632661.9846206277)
Generation 18: new best solution found!
[3, 0, 1, 1, 1, 3, 0, 0, 1, 1, 1, 1, 3, 1, 1, 1, 1, 0, 0, 1, 3, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0,

[[1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 (5891915.107878157, 184.0, 118.0, 658522.4531245104)]

In [36]:
#'scale_civs' produces 67% of 'rush' IC, but has 4% more mils and 142% more civs
genetic_algorithm(57,100, 60, 0.9, 'scale_civs', 'half year')

Generation 0: new best solution found!
[1, 0, 3, 3, 1, 1, 1, 1, 2, 3, 3, 2, 1, 3, 2, 0, 2, 2, 1, 0, 0, 1, 1, 0, 3, 0, 3, 0, 0, 0, 2, 0, 0, 1, 2, 1, 3, 0, 0, 3, 1, 0, 0, 2, 0, 1, 3, 2, 3, 1, 3, 1, 1, 3, 1, 3, 0] (823147.3093544266, 92.0, 86.0, 578201.4746317948)
Generation 0: new best solution found!
[2, 1, 2, 0, 3, 2, 1, 1, 0, 0, 2, 1, 1, 0, 3, 2, 2, 0, 1, 0, 0, 0, 3, 1, 3, 0, 3, 1, 3, 0, 1, 3, 2, 0, 0, 3, 1, 0, 1, 0, 3, 1, 0, 3, 1, 3, 2, 0, 1, 1, 1, 2, 1, 1, 0, 1, 2] (965185.47473083, 104.0, 92.0, 550164.2636001287)
Generation 1: new best solution found!
[1, 0, 3, 3, 1, 2, 1, 1, 0, 2, 0, 1, 3, 3, 1, 0, 1, 2, 0, 1, 0, 2, 1, 2, 1, 0, 1, 3, 3, 2, 3, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 3, 3, 0, 0, 3, 0, 1, 0, 1, 2, 3, 1, 0, 0, 0, 2] (1058944.380432072, 117.0, 89.0, 575445.9899194364)
Generation 1: new best solution found!
[1, 1, 1, 0, 1, 0, 3, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 3, 1, 2, 1, 1, 3, 0, 3, 1, 3, 0, 1, 3, 2, 0, 0, 3, 1, 0, 1, 0, 3, 1, 0, 3, 1, 3, 2, 0, 1, 1, 1, 2, 1, 1, 0, 1, 2] (12

Generation 17: new best solution found!
[1, 1, 1, 0, 1, 0, 3, 3, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 3, 1, 1, 3, 0, 3, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0] (2199959.0288074804, 142.0, 152.0, 582752.7089310125)
Generation 18: new best solution found!
[1, 1, 1, 0, 1, 0, 3, 0, 3, 1, 0, 0, 3, 1, 0, 0, 0, 1, 1, 0, 1, 1, 3, 1, 0, 0, 0, 3, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0] (2204511.6887124977, 142.0, 145.0, 607488.6691067687)
Generation 18: new best solution found!
[1, 1, 1, 0, 1, 0, 3, 3, 3, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 3, 1, 3, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0] (2205939.6784979394, 143.0, 140.0, 623739.1864071582)
Generation 19: new best solution found!
[1, 1, 1, 0, 1, 0, 3, 3, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 3, 1, 1, 3, 0, 3, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0,

Smoothing : new best solution found!
[1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0] (2492231.9937166073, 163.0, 161.0, 576977.5585357023)


[[1,
  1,
  1,
  1,
  1,
  3,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0],
 (2492231.9937166073, 163.0, 161.0, 576977.5585357023)]

In [37]:
genetic_algorithm(57,100, 60, 0.9, 'rush', 'half year')

Generation 0: new best solution found!
[3, 1, 3, 2, 1, 2, 0, 2, 1, 2, 0, 2, 0, 1, 1, 0, 0, 3, 1, 2, 2, 1, 0, 3, 2, 1, 3, 0, 2, 2, 3, 2, 1, 2, 3, 1, 0, 3, 2, 1, 3, 2, 2, 2, 0, 2, 0, 0, 2, 2, 1, 1, 0, 0, 1, 3, 3] (614973.8281299358, 104.0, 45.0)
Generation 0: new best solution found!
[2, 1, 3, 2, 3, 0, 0, 3, 2, 2, 2, 0, 2, 3, 0, 1, 1, 0, 3, 1, 2, 1, 3, 0, 2, 2, 0, 3, 1, 0, 1, 2, 0, 2, 1, 2, 3, 2, 2, 1, 2, 2, 1, 0, 2, 1, 1, 1, 1, 3, 2, 0, 1, 0, 2, 0, 3] (623385.6407044434, 105.0, 42.0)
Generation 0: new best solution found!
[0, 1, 2, 1, 1, 2, 0, 1, 3, 0, 3, 0, 1, 1, 3, 1, 1, 1, 2, 1, 1, 0, 0, 2, 0, 0, 2, 0, 0, 1, 3, 0, 2, 3, 2, 0, 1, 2, 1, 2, 3, 3, 0, 2, 0, 2, 0, 2, 1, 1, 1, 1, 2, 0, 0, 3, 2] (623721.5188400892, 121.0, 53.0)
Generation 0: new best solution found!
[3, 2, 0, 3, 3, 2, 1, 0, 3, 0, 2, 2, 0, 1, 1, 0, 2, 0, 2, 2, 3, 0, 1, 0, 0, 2, 1, 1, 0, 2, 1, 0, 0, 3, 0, 2, 1, 3, 3, 0, 2, 1, 0, 0, 2, 0, 3, 1, 1, 2, 2, 2, 2, 2, 3, 3, 1] (634909.8672590718, 113.0, 46.0)
Generation 0: new best s

Generation 22: new best solution found!
[2, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 1, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 3, 3, 1] (762262.6686533903, 138.0, 32.0)
Generation 22: new best solution found!
[1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 0, 2, 2, 2, 3, 2] (767532.7184519003, 143.0, 33.0)
Generation 24: new best solution found!
[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 1] (771509.51890404, 144.0, 33.0)
Generation 26: new best solution found!
[1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 1] (771531.1375417303, 144.0, 32.0)
Generation 26: new bes

[[1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  2,
  2,
  2,
  2,
  3,
  2],
 (794239.6159132903, 157.0, 36.0)]

In [38]:
genetic_algorithm(57,100, 60, 0.9, 'mils', 'half year')

Generation 0: new best solution found!
[3, 1, 0, 2, 0, 0, 1, 2, 1, 2, 1, 1, 3, 0, 0, 3, 0, 0, 3, 0, 1, 0, 2, 2, 2, 0, 3, 0, 1, 2, 3, 2, 3, 1, 2, 0, 0, 3, 0, 3, 2, 1, 0, 0, 2, 0, 0, 2, 0, 1, 0, 1, 0, 2, 1, 2, 3] (118.0, 51.0, 625008.5851495398)
Generation 0: new best solution found!
[1, 3, 3, 3, 3, 1, 3, 2, 1, 1, 1, 0, 0, 3, 2, 2, 1, 2, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 3, 0, 1, 2, 0, 1, 1, 0, 2, 0, 1, 1, 2, 2, 1, 2, 0, 3, 2, 0, 3, 3, 0, 2] (125.0, 57.0, 634072.5549572977)
Generation 1: new best solution found!
[3, 0, 2, 2, 2, 2, 3, 3, 1, 0, 1, 0, 3, 1, 0, 3, 1, 2, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 3, 0, 1, 2, 0, 1, 1, 0, 2, 0, 1, 1, 2, 2, 1, 2, 0, 3, 2, 0, 3, 3, 0, 2] (128.0, 61.0, 637082.7799476299)
Generation 2: new best solution found!
[1, 1, 1, 0, 3, 1, 3, 3, 3, 2, 3, 1, 1, 3, 3, 3, 0, 2, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 3, 0, 1, 2, 0, 1, 1, 0, 2, 0, 1, 1, 2, 2, 1, 2, 0, 3, 2, 0, 3, 3, 0, 2] (129.0, 65.0, 592093.9912869309)
Generation 2: new best s

Generation 17: new best solution found!
[1, 0, 1, 0, 2, 3, 0, 0, 3, 1, 3, 1, 3, 1, 1, 1, 1, 3, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 2, 0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 2, 0, 2, 2] (171.0, 71.0, 633242.0246471692)
Generation 17: new best solution found!
[1, 3, 1, 3, 0, 0, 1, 0, 3, 1, 3, 0, 1, 1, 1, 1, 3, 3, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 2, 0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 2, 0, 2, 2] (171.0, 73.0, 627021.2904128096)
Generation 18: new best solution found!
[1, 3, 1, 3, 0, 0, 1, 0, 3, 1, 3, 0, 1, 1, 1, 1, 3, 3, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 2, 0, 2, 2, 2, 2, 0, 0, 1, 2, 0, 2, 2] (172.0, 67.0, 637633.0064481468)
Generation 19: new best solution found!
[1, 1, 1, 1, 3, 0, 1, 0, 3, 1, 3, 0, 1, 1, 1, 1, 3, 3, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 2, 0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 1, 2, 0, 2, 2] (172.0, 72.0, 628729.6741308374)
Generation 19: new b

Generation 44: new best solution found!
[1, 3, 1, 3, 3, 1, 1, 3, 1, 3, 1, 3, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2] (197.0, 92.0, 572010.677347923)
Generation 45: new best solution found!
[1, 3, 1, 3, 3, 1, 1, 3, 1, 3, 1, 3, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2] (197.0, 97.0, 584108.1757958196)
Generation 47: new best solution found!
[1, 3, 1, 3, 3, 1, 1, 3, 1, 3, 1, 3, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2] (198.0, 95.0, 532510.641816941)
Generation 48: new best solution found!
[1, 0, 1, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2] (199.0, 89.0, 541294.4667924152)
Generation 49: new bes

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  3,
  3,
  3,
  3,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  2,
  2,
  2],
 (205.0, 96.0, 549657.0633062413)]

In [39]:
#The delay in research does not cause that big of a production loss
#Especially interesting is that going one year ahead is only marginally better
#than half a year, good to know in actual games

test = [0] * 57

(production(test, 'rush', 'one year'), production(test, 'rush', 'half year'), \
 production(test, 'rush', 'current'), production(test, 'rush', 'test'))

((804759.3246453318, 163.0, 74),
 (791767.0442336968, 162.0, 74),
 (765962.3847824389, 161.0, 74),
 (408398.71125558834, 135.0, 74))

In [40]:
#Interesting, it 'prebuilds' a military factory at the start

test = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
production(test, 'scale_civs', 'half year')

(2493391.7423907323, 171.0, 151.0, 590444.7687307968)